In [14]:
# Importing libraries
import pandas as pd
def recommendation():
    
    # Read csv file into a pandas dataframe
    df = pd.read_csv("books.csv")
    df = df[df.language_code =='eng']
    df['Title'] = df['title'].str.split('(').str[0]
    print("Enter rating w/wo <rating> or random for book suggestion")
    search= input("enter to search ")
    if search=="random":
        df_new= df.sample(replace=True)[['Title', 'authors',"rating"]]
        print("Title: ", df_new[['Title']].to_string(index=False, header=False))
        print("Author: ", df_new[['authors']].to_string(index=False, header=False))
        print("Rating: ", df_new[['rating']].to_string(index=False, header=False))
        return "Enjoy!"
    elif "rating" in search:
            rating=input("enter the range of rating First: the maximum rating Second: the minimum rating")
            rating=rating.split()
            rate1=float(rating[0])
            rate2=float(rating[1])
            df_rate = df[df['rating'].astype(float).between(rate1,rate2, inclusive = True)]
            df_new= df_rate.sample(replace=True)[['Title', 'authors',"rating"]]
            print("Title: ", df_new[['Title']].to_string(index=False, header=False))
            print("Author: ", df_new[['authors']].to_string(index=False, header=False))
            print("Rating: ", df_new[['rating']].to_string(index=False, header=False))
            return "Enjoy!"
print(recommendation())

Enter rating w/wo <rating> or random for book suggestion
enter to search rating 5
Title:   The Goon Show  Volume 4: My Knees Have Fallen Off!
Author:   NOT A BOOK
Rating:   5
Enjoy!
